In [1]:
import numpy as np
from datetime import datetime
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from lib.ChextXRayImages import *
from models.CustomPneumonia import CustomPneumoniaNN

import os, os.path
from PIL import Image
import copy

import torch.optim as optim
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor, ToPILImage
import torchvision.models as models

from torchsummary import summary

%matplotlib inline

In [2]:
# We need to set the working directory since we are using relative paths from various locations
os.chdir('C:\git\Springboard-Public\Capstone Project 2')

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print(f'Working on device={device}')

Working on device=cuda


In [4]:
net = CustomPneumoniaNN()

net = nn.DataParallel(net)
net.to(device)

summary(net, (1, 320, 320))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [-1, 512, 320, 320]          13,312
       BatchNorm2d-2        [-1, 512, 320, 320]           1,024
         MaxPool2d-3        [-1, 512, 160, 160]               0
            Conv2d-4        [-1, 256, 160, 160]       1,179,904
       BatchNorm2d-5        [-1, 256, 160, 160]             512
         MaxPool2d-6          [-1, 256, 80, 80]               0
            Conv2d-7           [-1, 64, 80, 80]         147,520
       BatchNorm2d-8           [-1, 64, 80, 80]             128
         MaxPool2d-9           [-1, 64, 40, 40]               0
           Linear-10                 [-1, 1024]     104,858,624
          Dropout-11                 [-1, 1024]               0
           Linear-12                  [-1, 512]         524,800
          Dropout-13                  [-1, 512]               0
           Linear-14                   

In [5]:
loaders = Loaders()
batch_size=32
train_percent=0.85
number_images = 15000
train_loader, val_loader = loaders.getDataTrainValidateLoaders(batch_size=batch_size, 
                                                                        train_percent=train_percent, 
                                                                        n_random_rows=number_images)
print(f'Number of Training Batches: {len(train_loader):,}')
print(f'Number of Validation Batches: {len(val_loader):,}')
print(f'Number of Training Images: {len(train_loader) * batch_size:,}')
print(f'Number of Validation Images: {len(val_loader) * batch_size:,}')

Number of Training Batches: 399
Number of Validation Batches: 71
Number of Training Images: 12,768
Number of Validation Images: 2,272


In [6]:
learning_rate = 1e-4
num_epochs = 30

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=learning_rate)#, weight_decay=0.9)

In [7]:
train_accuracy_index = []
train_acc, train_total, train_correct = 0, 0, 0
val_accuracy_index = []
val_acc, val_total, val_correct = 0, 0, 0
test_accuracy_index = []
test_acc, test_total, test_correct = 0, 0, 0
losses_index = []
for epoch in range(num_epochs):  # loop over the dataset multiple times
    start_time = datetime.now()
    net.train()
    running_loss = 0.0
    epoch_loss = 0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data['img'], data['labels']['Pneumonia']
        # move data to device GPU OR CPU
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        
        #Get Training Accuracty
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        train_acc = train_correct/train_total
        train_accuracy_index.append(train_acc)

        #loss, back prop and update params
        loss = criterion(outputs, labels)#.float())
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    epoch_loss = epoch_loss / len(train_loader)
    time_elapsed = datetime.now() - start_time
    losses_index.append(epoch_loss)
    
    # Validation set
    net.eval()
    with torch.no_grad():
      for data in val_loader:
          inputs, labels = data
          inputs, labels = inputs.to(device), labels.to(device)
          outputs = net(inputs)
          _, predicted = torch.max(outputs.data, 1)
          val_total += labels.size(0)
          val_correct += (predicted == labels).sum().item()
          val_acc = val_correct/val_total
      val_accuracy_index.append(val_acc)

      for data in val_loader:
          inputs, labels = data
          inputs, labels = inputs.to(device), labels.to(device)
          outputs = net(inputs)
          _, predicted = torch.max(outputs.data, 1)
          test_total += labels.size(0)
          test_correct += (predicted == labels).sum().item()
          test_acc = test_correct/test_total
      test_accuracy_index.append(test_acc)
    
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Epoch Loss: {epoch_loss:.4f} Training Accuracy: {train_acc:.4f}  Validation Accuracy: {val_acc:.4f} Test Accuracy: {test_acc:.4f} - (time={time_elapsed})')

RuntimeError: CUDA out of memory. Tried to allocate 6.25 GiB (GPU 0; 8.00 GiB total capacity; 420.64 MiB already allocated; 5.54 GiB free; 422.00 MiB reserved in total by PyTorch)